# Week 8 Encoder Decoder

In [1]:
import numpy as np
import tensorflow as tf
keras = tf.keras


In [2]:
!!curl -O http://www.manythings.org/anki/hin-eng.zip
!!unzip hin-eng.zip


['Archive:  hin-eng.zip',
 '  inflating: hin.txt                 ',
 '  inflating: _about.txt              ']

In [3]:
batch_size = 64
epochs = 100
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000

data_path = "hin.txt"


## Perform required text pre-processing

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


Number of samples: 2979

Number of unique input tokens: 70

Number of unique output tokens: 90

Max sequence length for inputs: 107

Max sequence length for outputs: 123


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


## Train, Validate, Test, and compare the two encoder-decoder models, each containing the following, to translate English sentences to Hindi.

### a) LSTM Model

In [17]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [8]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)


Epoch 1/100

38/38 [==============================] - 10s 68ms/step - loss: 1.2964 - accuracy: 0.7848 - val_loss: 1.7195 - val_accuracy: 0.6848

Epoch 2/100

38/38 [==============================] - 1s 28ms/step - loss: 1.0091 - accuracy: 0.8069 - val_loss: 1.6166 - val_accuracy: 0.6857

Epoch 3/100

38/38 [==============================] - 1s 27ms/step - loss: 0.8965 - accuracy: 0.8086 - val_loss: 1.6068 - val_accuracy: 0.6873

Epoch 4/100

38/38 [==============================] - 1s 27ms/step - loss: 0.8542 - accuracy: 0.8091 - val_loss: 1.5557 - val_accuracy: 0.6873

Epoch 5/100

38/38 [==============================] - 1s 27ms/step - loss: 0.8226 - accuracy: 0.8090 - val_loss: 1.3669 - val_accuracy: 0.6873

Epoch 6/100

38/38 [==============================] - 1s 27ms/step - loss: 0.8101 - accuracy: 0.8089 - val_loss: 1.4614 - val_accuracy: 0.6873

Epoch 7/100

38/38 [==============================] - 1s 27ms/step - loss: 0.7917 - accuracy: 0.8085 - val_loss: 1.3538 - val_accuracy:

In [9]:
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence


In [10]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


1/1 [==============================] - 0s 341ms/step

1/1 [==============================] - 0s 336ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 21ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 21ms/step

1/1 [==============================] - 0s 22ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 20

### b) GRU Model

In [55]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.GRU(latent_dim, return_state=True)
encoder_outputs, state_c = encoder(encoder_inputs)

encoder_states = [state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Model: "model_44"

__________________________________________________________________________________________________

 Layer (type)                Output Shape                 Param #   Connected to                  


 input_33 (InputLayer)       [(None, None, 70)]           0         []                            

                                                                                                  

 input_34 (InputLayer)       [(None, None, 90)]           0         []                            

                                                                                                  

 gru_4 (GRU)                 [(None, 256),                251904    ['input_33[0][0]']            

                              (None, 256)]                                                        

                                                                                                  

 gru_5 (GRU)                 [(None, None, 256),          267264    ['input_34[0

In [56]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)


Epoch 1/100

38/38 [==============================] - 5s 50ms/step - loss: 1.3827 - accuracy: 0.7850 - val_loss: 1.6951 - val_accuracy: 0.6848

Epoch 2/100

38/38 [==============================] - 1s 26ms/step - loss: 0.9574 - accuracy: 0.8064 - val_loss: 1.5486 - val_accuracy: 0.6848

Epoch 3/100

38/38 [==============================] - 1s 26ms/step - loss: 0.9020 - accuracy: 0.8065 - val_loss: 1.4771 - val_accuracy: 0.6848

Epoch 4/100

38/38 [==============================] - 1s 26ms/step - loss: 0.8397 - accuracy: 0.8084 - val_loss: 1.3589 - val_accuracy: 0.6873

Epoch 5/100

38/38 [==============================] - 1s 27ms/step - loss: 0.7973 - accuracy: 0.8066 - val_loss: 1.3066 - val_accuracy: 0.6870

Epoch 6/100

38/38 [==============================] - 1s 29ms/step - loss: 0.7562 - accuracy: 0.8120 - val_loss: 1.2376 - val_accuracy: 0.6932

Epoch 7/100

38/38 [==============================] - 1s 33ms/step - loss: 0.7323 - accuracy: 0.8179 - val_loss: 1.2022 - val_accuracy: 

In [57]:
encoder_inputs = model.input[0]
encoder_outputs, state_c_enc = model.layers[2].output
encoder_states = [state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_c]
decoder_gru = model.layers[3]
decoder_outputs, state_c_dec = decoder_gru(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    # states_value = np.reshape(states_value, (1, 1, 1, 90))

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    print("hello")
    while not stop_condition:

        output_tokens, c = decoder_model.predict([target_seq])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char


        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        print(target_seq.shape)
        target_seq[0, 0, sampled_token_index] = 1.0


        target_seq = np.reshape(target_seq, (1, 1, num_decoder_tokens))

        states_value = [c]
    return decoded_sentence


In [ ]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(decoded_sentence)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
